<a href="https://colab.research.google.com/github/stepby9/tableau_pptx_slides_automation/blob/main/Tableau_to_pptx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1.** Click the "Run" button and wait for the script to finish.

This script deletes all images and add new ones from tableau. You can adjust the position/size of the images in the "VIEWS_TO_INSERT" section in the code.




In [ ]:
# @title Run here
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install required packages
!pip install -q python-pptx requests

import requests
import xml.etree.ElementTree as ET
from io import BytesIO
from pptx import Presentation
from pptx.util import Inches
from pptx.enum.shapes import MSO_SHAPE_TYPE

# === CONFIGURATION ===

# Path to your PPTX template
PPTX_PATH = "/content/drive/Shareddrives/xxx/yyy.pptx"

# Tableau credentials & default workbook
TABLEAU_SERVER   = "https://10az.online.tableau.com"
API_VERSION      = "3.12"
PAT_NAME         = "tableau_token_name"
PAT_SECRET       = "tableau_token_secret"
SITE_CONTENT_URL = "your_site_url"

# Default workbook (used for slides 3–6)
WORKBOOK_MAIN    = "tableau-workbook-id"
# New workbook for slides 8–9
WORKBOOK_RENEW   = "another-tableau-workbook-id"

# Each entry describes one image to insert:
# - name:          Tableau view name
# - workbook:      which workbook ID to use
# - slide_index:   zero-based slide index (slide 3 → index=2, etc.)
# - left, top:     position in inches
# - width,height:  size in inches
# - params:        optional Tableau URL parameters (e.g. filters / parameters)
# - crop_top:      inches to crop from top (optional)
VIEWS_TO_INSERT = [
    # original four
    {"name":"Current Quote Delivered Term Mix","workbook":WORKBOOK_MAIN,"slide_index":2,
     "left":0.38,"top":0.86,"width":9.47,"height":4.5},
    {"name":"First Quote Delivered Term Mix","workbook":WORKBOOK_MAIN,"slide_index":3,
     "left":0.49,"top":1.22,"width":8.57,"height":3.48},
    {"name":"Open Opportunities","workbook":WORKBOOK_MAIN,"slide_index":4,
     "left":1.96,"top":0.85,"width":6.58,"height":4.5},
    {"name":"Expiration Mix","workbook":WORKBOOK_MAIN,"slide_index":5,
     "left":0.54,"top":1.25,"width":8.67,"height":4.5},
    # two new from the renewals workbook
    {"name":"Renewals Rate","workbook":WORKBOOK_RENEW,"slide_index":7,
     "left":0.35,"top":0.86,"width":9.47,"height":4.5,
     "params":{"Metric Toggle":"Annualized (M$)"},
     "crop_top":0.7},
    {"name":"Renewals Rate","workbook":WORKBOOK_RENEW,"slide_index":8,
     "left":0.34,"top":0.86,"width":9.47,"height":4.5,
     "params":{"Metric Toggle":"# Arrays"},
     "crop_top":0.7},
]

NAMESPACES = {"t": "http://tableau.com/api"}

# === FUNCTIONS ===

def delete_all_pictures(pptx_path):
    prs = Presentation(pptx_path)
    for slide in prs.slides:
        for shape in list(slide.shapes):
            if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
                slide.shapes._spTree.remove(shape._element)
    prs.save(pptx_path)

def insert_view_image(cfg):
    """
    Signs into Tableau, fetches the image for cfg["name"] from cfg["workbook"],
    inserts it on slide cfg["slide_index"], crops, sends to back, then signs out.
    Errors are caught and printed; the function returns after handling each view.
    """
    view_name   = cfg["name"]
    wb_id       = cfg["workbook"]
    si          = cfg["slide_index"]
    left_in, top_in   = cfg["left"], cfg["top"]
    w_in, h_in        = cfg["width"], cfg["height"]
    params_override   = cfg.get("params", {})
    crop_top_inches   = cfg.get("crop_top", 0)

    token = None
    try:
        # --- Sign in ---
        signin = requests.post(
            f"{TABLEAU_SERVER}/api/{API_VERSION}/auth/signin",
            data=f"""
            <tsRequest>
              <credentials personalAccessTokenName="{PAT_NAME}"
                           personalAccessTokenSecret="{PAT_SECRET}">
                <site contentUrl="{SITE_CONTENT_URL}" />
              </credentials>
            </tsRequest>""",
            headers={"Content-Type":"text/xml"}
        )
        signin.raise_for_status()
        root    = ET.fromstring(signin.text)
        token   = root.find(".//t:credentials", NAMESPACES).attrib["token"]
        site_id = root.find(".//t:site", NAMESPACES).attrib["id"]
        auth_h  = {"X-Tableau-Auth": token}

        # --- Get list of views ---
        vr = requests.get(
            f"{TABLEAU_SERVER}/api/{API_VERSION}/sites/{site_id}/workbooks/{wb_id}/views",
            headers=auth_h
        )
        vr.raise_for_status()
        views = {v.attrib["name"]:v.attrib["id"]
                 for v in ET.fromstring(vr.text).findall(".//t:view",NAMESPACES)}

        if view_name not in views:
            print(f"⚠️ View not found in workbook {wb_id!r}: '{view_name}'. SKIPPING.")
            return

        view_id = views[view_name]

        # --- Download image with optional parameters ---
        params = {"maxWidth":1920,"maxHeight":1080}
        for k,v in params_override.items():
            params[f"vf_{k}"] = v

        ir = requests.get(
            f"{TABLEAU_SERVER}/api/{API_VERSION}/sites/{site_id}/views/{view_id}/image",
            headers=auth_h, params=params
        )
        ir.raise_for_status()
        img_stream = BytesIO(ir.content)

        # --- Insert into PPT ---
        prs   = Presentation(PPTX_PATH)
        slide = prs.slides[si]
        pic   = slide.shapes.add_picture(
            img_stream,
            left=Inches(left_in),
            top=Inches(top_in),
            width=Inches(w_in),
            height=Inches(h_in)
        )

        # Crop top if requested
        if crop_top_inches > 0:
            # crop_top expects fraction of total height
            frac = crop_top_inches / h_in
            pic.crop_top = frac

        # send to back
        slide.shapes._spTree.insert(2, pic._element)
        prs.save(PPTX_PATH)

        print(f"✅ Inserted '{view_name}' on slide {si+1}")

    except Exception as e:
        print(f"❌ Error inserting '{view_name}' on slide {si+1}: {e}")

    finally:
        # Sign out if we signed in
        if token:
            try:
                requests.post(
                    f"{TABLEAU_SERVER}/api/{API_VERSION}/auth/signout",
                    headers={"X-Tableau-Auth":token}
                )
            except:
                pass

# === EXECUTION ===

print("🧹 Clearing existing images…")
delete_all_pictures(PPTX_PATH)

for cfg in VIEWS_TO_INSERT:
    insert_view_image(cfg)

print("🏁 Finished all inserts.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🧹 Clearing existing images…
✅ Inserted 'Current Quote Delivered Term Mix' on slide 3
✅ Inserted 'First Quote Delivered Term Mix' on slide 4
✅ Inserted 'Open Opportunities' on slide 5
✅ Inserted 'Expiration Mix' on slide 6
✅ Inserted 'Renewals Rate' on slide 8
✅ Inserted 'Renewals Rate' on slide 9
🏁 Finished all inserts.
